In [ ]:
from tf_lab.external.structural_pc_losses import losses

nn_distance, approx_match, match_cost = losses()

In [ ]:
LOAD AE

In [ ]:
test_recon, l_test = reconstruct_pclouds(ae, in_data['test'].point_clouds, batch_size=100)

In [ ]:
worst_k = 224

In [ ]:
cost_p1_p2, gt_mapped_to, _, _ = nn_distance(ae.gt, ae.x_reconstr)

worst_mapped = tf.nn.top_k(cost_p1_p2, worst_k)

b_size = tf.shape(gt_mapped_to)[0]

row_idx = tf.range(b_size, dtype=tf.int32)
row_idx = tf.reshape(tf.tile(row_idx, [worst_k]), [worst_k, -1])
row_idx = tf.transpose(row_idx)
col_idx = worst_mapped.indices

coords = tf.transpose(tf.stack([row_idx, col_idx]))

gt_problematics = tf.squeeze(tf.gather_nd(ae.x, coords)) # worse points-of GT

col_idx = tf.transpose(tf.gather_nd(gt_mapped_to, coords)) 
coords = tf.transpose(tf.stack([row_idx, col_idx]))
col_idx = tf.squeeze(tf.gather_nd(ae.x_reconstr, coords)) # Guys where above where matched

In [ ]:
rid = np.random.randint(in_data['train'].num_examples-1)
feed = in_data['train'].point_clouds[rid:rid+1]
feed_dict = {ae.x: feed}
where_gt_was_mapped, gt_prob_, recon = ae.sess.run([col_idx, gt_problematics, ae.x_reconstr[0]], feed_dict=feed_dict)
fig1 = Point_Cloud(feed[0]).plot(azim=15);
fig2 = Point_Cloud(recon).plot(azim=15);
Point_Cloud(where_gt_was_mapped).plot(azim=15);
Point_Cloud(gt_prob_).plot(azim=15);

In [ ]:
# Use below to verify.
from pc_completions.evaluation import accuracy_of_pc_completion
# _, color = accuracy_of_pc_completion(feed[0], recon,thres=0.02, ret_dists=True)
_, color = accuracy_of_pc_completion(recon, feed[0],thres=0.02, ret_dists=True)
color = np.squeeze(color)
rev_idx = np.argsort(color)[::-1]
Point_Cloud(recon[rev_idx[:224]]).plot(azim=15)